# Importing the libraries

In [1]:
from langchain.llms.openai import OpenAIChat
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.globals import set_debug
from dotenv import load_dotenv

set_debug(True)  # Enable debug mode
load_dotenv()  # Load the variables from .env file

True

# Models

In this example we will use OpenAI's LLM Models to generate text, But you can use any LLM you want from the supported llms within langchain e.i. HuggingFace, LLaMa, PaLM, and etc.





Specifically we will use the newer models that are connected to `v1/chat/completion` endpoint, and can be used in `OpenAIChat` class. 

for more information about the models and the endpoint please refer to [OpenAI's API Docs](https://platform.openai.com/docs/guides/text-generation/chat-completions-vs-completions)

To setup your OpenAI API key: 
1. Create an OpenAI account
2. Create a new API key from this [link](https://platform.openai.com/api-keys)
3. Copy the API key and paste it in the `OPENAI_API_KEY` variable in `.env` file



In [2]:
# Temperature: a number between 0 and 1. The closer to 0,\
# the more predictable the text is, the closer to 1, the more Hallucinations.

model = OpenAIChat(
    model="gpt-3.5-turbo-1106",
    temperature=0.0,
)

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [3]:
# To make sure the model is working, we can test it with a prompt.

response = model("Hello, how are you?")

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Hello, how are you?"
  ]
}
[llm/end] [1:llm:OpenAIChat] [2.22s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello! I'm just a language model AI, so I don't have feelings, but I'm here to help you. How can I assist you today?",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 13,
      "completion_tokens": 32,
      "total_tokens": 45
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}


In [4]:
response

"Hello! I'm just a language model AI, so I don't have feelings, but I'm here to help you. How can I assist you today?"

# Prompts

When building apps you need the model to generate output in specific format each inference, so when building a prompt you need to follow some prompt engineering techniques, which are discussed in [Deeplearning.ai's ChatGPT Prompt Engineering for Developers](https://learn.deeplearning.ai/chatgpt-prompt-eng) go check it out it's free.

I've created prompts that meets our needs in the examples that follows the guidelines provided in the short course, so don't worry about it being complicated.

In [ ]:
text = """
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

In [ ]:
# We will use f-strings to insert the text into the prompt.
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

In [ ]:
print(prompt)


Summarize the text delimited by triple backticks \ 
into a single sentence.
```
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
```



In [ ]:
# Getting the response from the model.
response = model(prompt)

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Summarize the text delimited by triple backticks \\ \ninto a single sentence.\n```\nYou should express what you want a model to do by \\ \nproviding instructions that are as clear and \\ \nspecific as you can possibly make them. \\ \nThis will guide the model towards the desired output, \\ \nand reduce the chances of receiving irrelevant \\ \nor incorrect responses. Don't confuse writing a \\ \nclear prompt with writing a short prompt. \\ \nIn many cases, longer prompts provide more clarity \\ \nand context for the model, which can lead to \\ \nmore detailed and relevant outputs.\n```"
  ]
}


[llm/end] [1:llm:OpenAIChat] [1.84s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Clear and specific instructions for a model will guide it towards the desired output and reduce the chances of receiving irrelevant or incorrect responses, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 134,
      "completion_tokens": 46,
      "total_tokens": 180
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}


Prompts sometimes (actually most of the time) will be long and contains lots of parameters, so Langchain provides a simple way to generate prompts for your LLMs, you can use the `PromptTemplate` class to generate prompts for your LLMs.

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text delimited by triple backticks, \
please extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

and the values for each key are the information extracted from the text.

```{text}```
"""

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["text"],
    template=review_template,
)

In [ ]:
prompt = prompt_template.format(text=customer_review)

In [ ]:
print(prompt)

For the following text delimited by triple backticks, please extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

and the values for each key are the information extracted from the text.

```This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out 

In [ ]:
response = model(prompt)

[llm/start] [1:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "For the following text delimited by triple backticks, please extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\nand the values for each key are the information extracted from the text.\n\n```This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morni

In [ ]:
print(response)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
